In [16]:
import pandas as pd
import numpy as np
import nanoid
import pymongo
from openai import OpenAI
import requests
from datetime import datetime



ModuleNotFoundError: No module named 'pymongo'

In [15]:
quantites = np.random.randint(1,100,size=48)
prices_buy = np.random.randint(1,2,size=48)*10 + np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1
prices_sale = prices_buy - np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1


In [18]:
data = pd.read_csv('data/flowershopdata.csv')
data['Description'] = data[['Description', 'Needs','Season']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
data['Sale_price'] = prices_buy
data['Buy_price'] = np.round(prices_sale,2)
data['Quantities'] = quantites
jpg_paths = [f"init_data/products/img_flowers/{flower.lower()}.jpg" for flower in data['Name']]
data['pic_path'] = jpg_paths
data['id'] = [nanoid.generate(size=10) for x in range(48)]
data['date'] =[datetime.now().strftime("%d-%m-%Y %H:%M:%S") for x in range(48)]
data['tags'] = ['flower' for x in range(48)]
data.drop(columns=['Needs','Season'],inplace=True)

In [19]:
data.to_csv('data/flowershopdata_clean.csv')

In [46]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def fetch_first_image_url(query):
    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    search_url = f"https://pl.pinterest.com/search/pins/?q={query}%20flower&rs=typed"
    driver.get(search_url)

    time.sleep(2)
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img')

    if len(image_elements) >= 10:
        # Get the URL of the 10th image
        image_element = image_elements[9]  # Index 9 corresponds to the 10th image (0-based index)
        image_url = image_element.get_attribute('src')
    else:
        print("Not enough images found on the page")
        image_url = None

    driver.quit()
    return image_url

def save_image_from_url(url, directory, filename):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image_path = os.path.join(directory, filename)
    image.save(image_path)
    print(f"Image saved to {image_path}")



In [6]:
import nanoid
def create_product(product_data):
    base_url = "http://127.0.0.1:8000"
    response = requests.post(f"{base_url}/products/", json=product_data)
    print(response.json())

In [7]:
data = pd.read_csv('data/flowershopdata_clean.csv').drop(columns=['Unnamed: 0'])



In [40]:
client = QdrantClient(url="http://127.0.0.1:6333")
names = [item.name for item in client.get_collections().collections]
print(names)

[]


In [32]:
# from sentence_transformers import SentenceTransformer, util
from qdrant_client import QdrantClient, AsyncQdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct

# model = SentenceTransformer('intfloat/e5-small-v2')
vec_len = 384
client = QdrantClient(url="http://127.0.0.1:6333")
names = [item.name for item in client.get_collections().collections]
if 'products_description' not in names:
    client.create_collection(
        collection_name="products_description",
        vectors_config=VectorParams(size=vec_len, distance=Distance.COSINE),)
else:
    print('Collection exits')


In [39]:
client.delete_collection("products_description")

False

In [8]:
points = [PointStruct(id=index, 
                      vector=model.encode(item['Description'], normalize_embeddings=True), 
                      payload={"Id": item['id'],
                               "Name": item['Name'],
                               "Description":item['Description']}) for index,item in data.iterrows()]


In [215]:
operation_info = client.upsert(
    collection_name="products_description",
    wait=True,
    points=points,
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [11]:
from pymongo import MongoClient
from openai import OpenAI
from typing import List
import pymongo
# from sentence_transformers import SentenceTransformer
from qdrant_client import AsyncQdrantClient, models


In [3]:
question = "I would like white flower that needs acidic soil"
# model = SentenceTransformer("intfloat/e5-small-v2")
bot = {
    "mongo_client": MongoClient("mongodb://localhost:27017/"),
    "openai_client":  OpenAI(api_key="sk-proj-8j1NyU5gvA4njucbGG3uT3BlbkFJxBLHcRn4z7futfYb4sgb")
},

In [5]:
client = AsyncQdrantClient("http://localhost:6333")

In [7]:
sad = await client.scroll(
    collection_name ="products_description",
    scroll_filter = models.Filter(
        must = [models.FieldCondition(
            key='name',
            match =models.MatchValue(
                value= 'Bloodroot'))
                ]
    ),
    with_vectors = True
)


In [8]:
sad[0][0]

Record(id=6, payload={'description': 'Named for its green colored sap that flows from cut stems, bloodroot flowers are white with a bright yellow center. Needs well-drained soil. Blooms in early spring.', 'id': 'AtfxNCsTNK', 'is_enabled': False, 'name': 'Bloodroot'}, vector=[-0.06801903, 0.023938809, 0.019408107, 0.02205197, 0.007889741, -0.013003591, 0.09313711, -0.031289045, -0.004206649, 0.053018305, 0.040470704, -0.037663784, -0.06471311, 0.0623793, 0.037090648, -0.011865469, -0.03745662, 0.044330344, -0.08975763, 0.073678724, 0.09839359, -0.029636282, -0.0021210266, -0.058417518, -0.048956174, 0.001813808, 0.064931795, 0.060150165, -0.058084752, -0.12609239, -0.04863415, 0.042198583, -0.013742634, -0.026881536, 0.05558968, -0.046346456, -0.0060306466, 0.06291792, 0.008927591, 0.043545894, -0.056614053, -0.084036484, 0.024140395, -0.020228786, -0.0062697656, -9.016218e-05, -0.035800233, -0.0274234, 0.05434272, 0.011605927, -0.04954425, -0.008654845, 0.046543993, 0.07357088, 0.04649

In [71]:
sad = await client.delete_collection(
    collection_name ="products_description"
)

In [78]:
requests.post("http://127.0.0.1:8000/products/vec_search/", json={'question': 'Red flower'}).json()


{'detail': 'Vector DB error'}

In [77]:
count = await client.count(collection_name="products_description")
count.count

0

In [18]:
# Connect to the MongoDB server
client_mongo = pymongo.MongoClient("mongodb://localhost:27017/")

# Select your database
db = client_mongo["ecommerce-app"]

# List all collection names in the database
collection_names = db.list_collection_names()


collection = db['products']
# Find all documents in the collection
documents = collection.find()
for x in documents:
    print(x)


# print(len(documents))




In [55]:
import requests
import numpy as np
url = 'http://127.0.0.1:8000/products'
def get_reccomendations(number):
    documents = requests.get(url).json()
    profit = [item.get('sell_price') - item.get('buy_price') for item in documents]
    profit = np.array(profit)/max(profit)*2/3
    quantites = [item.get('quantity') for item in documents]
    quantites = np.array(quantites)/max(quantites)*1/3
    score = profit+quantites
    names = [{"id":item.get('id'), "score":sc} for item,sc in zip(documents,score)]
    sorted_data = sorted(names, key=lambda x: x['score'], reverse=True)
    products = [requests.get(f'{url}/{item["id"]}').json() for item in sorted_data[:number]]
    return products

In [56]:
a = get_reccomendations(5)

In [57]:
a

[{'id': 'sF4-20cYQc',
  'name': 'Forsythia',
  'description': 'Early blooming shrub with bright yellow flowers that herald the arrival of spring. Needs well-drained soil. Blooms in early spring.',
  'sell_price': 15.1,
  'quantity': 80,
  'buy_price': 7.0,
  'date': 'None',
  'image_id': '666107be7c8302008d46923b',
  'tags': ['flower'],
  'is_enabled': True},
 {'id': '9cQPkJ5fnG',
  'name': 'Daisy',
  'description': 'Simple and charming, daisies are a symbol of innocence and purity, recognized by their white petals and yellow centers. Needs well-drained soil. Blooms in spring to autumn.',
  'sell_price': 16.0,
  'quantity': 93,
  'buy_price': 8.6,
  'date': 'None',
  'image_id': '666107bd7c8302008d469226',
  'tags': ['flower'],
  'is_enabled': True},
 {'id': 'CiXDjEv0vN',
  'name': 'Rhododendron',
  'description': 'Large clusters of blooms, often used in landscaping for their showy flowers and evergreen foliage. Needs acidic soil. Blooms in spring.',
  'sell_price': 18.7,
  'quantity':

In [47]:
products

[{'id': 'sF4-20cYQc',
  'name': 'Forsythia',
  'description': 'Early blooming shrub with bright yellow flowers that herald the arrival of spring. Needs well-drained soil. Blooms in early spring.',
  'sell_price': 15.1,
  'quantity': 80,
  'buy_price': 7.0,
  'date': 'None',
  'image_id': '666107be7c8302008d46923b',
  'tags': ['flower'],
  'is_enabled': True},
 {'id': '9cQPkJ5fnG',
  'name': 'Daisy',
  'description': 'Simple and charming, daisies are a symbol of innocence and purity, recognized by their white petals and yellow centers. Needs well-drained soil. Blooms in spring to autumn.',
  'sell_price': 16.0,
  'quantity': 93,
  'buy_price': 8.6,
  'date': 'None',
  'image_id': '666107bd7c8302008d469226',
  'tags': ['flower'],
  'is_enabled': True},
 {'id': 'CiXDjEv0vN',
  'name': 'Rhododendron',
  'description': 'Large clusters of blooms, often used in landscaping for their showy flowers and evergreen foliage. Needs acidic soil. Blooms in spring.',
  'sell_price': 18.7,
  'quantity':

In [13]:
db.drop_collection('products')
for document in documents:
    print(document)

In [6]:
from pydantic import BaseModel,Field
class Product(BaseModel):
    id: str = Field(default_factory=lambda: nanoid.generate(size=10))
    name: str = ''
    description: str = ''
    sell_price: float = 0
    quantity: int = 0
    buy_price: float = 0
    date: str = Field(default_factory=lambda: datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    image_id: str | None = None
    tags: List[str] = Field(default_factory=list)
    is_enabled: bool = True

class VecProduct(BaseModel):
    id: str = ''
    name: str = ''
    description: str = ''
    is_enabled: bool = True

In [34]:
from sentence_transformers import SentenceTransformer
from qdrant_client import AsyncQdrantClient
from qdrant_client.models import Filter, FieldCondition, PointStruct, MatchValue, PointVectors

class QdrantManager:
    qdrant_client: AsyncQdrantClient
    modelEmbed: SentenceTransformer

    def __init__(self) -> None:
        self.qdrant_client = AsyncQdrantClient('http://127.0.0.1:6333')
        self.modelEmbed = SentenceTransformer('intfloat/e5-small-v2', cache_folder='src/model_st')
        pass

    async def search(self, query:str):
        vector = self.modelEmbed.encode(query, normalize_embeddings=True)
        response = await self.qdrant_client.search(collection_name="products_description",
                                          query_vector=vector,
                                          limit=3,
                                          query_filter= Filter(
                                              must=[
                                                  FieldCondition(
                                                      key='is_enabled',
                                                      match = MatchValue(value=True)
                                                  )
                                              ]
                                          ))
        return response

    async def add_product_vec(self,product: Product):
        if not await self.__check_db__(product.name):
            vector = self.modelEmbed.encode(product.description,normalize_embeddings=True)
            num = await self.qdrant_client.count(collection_name="products_description")
            point = PointStruct(
                        id=int(num.count),
                        vector=vector,
                        payload={"id": product.id, "name": product.name, 
                                 "description": product.description, 
                                 "is_enabled": product.is_enabled}
                    )

            await self.qdrant_client.upsert(
                collection_name="products_description",
                wait=True,
                points=[point])
        else:
            print('Product already exists')

    async def update_product(self,product: Product):
        product_in_base = await self.__check_db__(product.name)
        if product.is_enabled:
            updated_product = VecProduct(**product_in_base.payload)
            updated_product.description = product.description
            vector = self.modelEmbed.encode(product.description,normalize_embeddings=True)
            # point = PointVectors(
            #         id=int(product_in_base.id),
            #         vector=vector,
            #         payload=dict(updated_product)
            #     )
            payload=dict(updated_product)
            await self.qdrant_client.update_vectors(
                collection_name='products_description',
                points = [
                    PointVectors(
                    id=int(product_in_base.id),
                    vector=vector,
                )
                ]
            )
        else:
            updated_product = VecProduct(**product_in_base.payload)
            updated_product.is_enabled = product.is_enabled
            # point = PointVectors(
            #         id=int(product_in_base.id),
            #         vector=product_in_base.vector,
            #         payload=dict(updated_product)
            # )
            payload=dict(updated_product)
        await self.qdrant_client.overwrite_payload(
            collection_name="products_description",
            wait = True,
            points=Filter(must=[FieldCondition(
                            key='name',
                            match = MatchValue(value=updated_product.name)
            )]),
            payload=payload
        )

    async def __check_db__(self,name):
        record = await self.qdrant_client.scroll(
        collection_name ="products_description",
        scroll_filter = Filter(
            must = [FieldCondition(
                key='name',
                match =MatchValue(
                    value= name))
                    ]
                ),
                with_vectors = True
            )
        if record[0]:
            return record[0][0]
        else:
            return False



In [35]:
dwa = QdrantManager()

In [36]:
example = Product()
example.buy_price = 1
example.name = 'Kwiat'
example.description = 'szesc dwa'
example.sell_price = 1
example.buy_price = 1
example.quantity = 5
example.tags = ['dwa']


In [40]:
await dwa.search('szesc')

[ScoredPoint(id=0, version=5, score=0.9322207, payload={'description': 'szesc dwa', 'id': 'ghzCyW6g4q', 'is_enabled': True, 'name': 'Kwiat'}, vector=None, shard_key=None)]

In [38]:
await dwa.add_product_vec(example)

Product already exists


In [39]:
await dwa.update_product(example)

In [135]:
qdrant_client = QdrantClient('localhost:6333')
results = qdrant_client.search(
    collection_name="products_description",
    query_vector=list(model.encode('dwa', normalize_embeddings=True)),
    limit=3,
)
type(results[0])

qdrant_client.http.models.models.ScoredPoint

In [52]:
model[1].word_embedding_dimension

384

In [16]:
count = await client.count(collection_name='products_description')

print(count)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `products_description` doesn\'t exist!"},"time":0.000034792}'

In [15]:
await client.delete_collection(collection_name='products_description')

True

In [55]:
import requests


url = 'http://127.0.0.1:8000/products'
response = requests.get(f'{url}/').json()
[[item.get('buy_price')-item.get('sell_price'),item.get('quantity')] for item in response]

[[4.1, 40],
 [0.5999999999999996, 67],
 [4.800000000000001, 41],
 [8.3, 67],
 [8.2, 36],
 [4.499999999999998, 2],
 [7.200000000000001, 70],
 [8.5, 11],
 [8.7, 49],
 [0.1999999999999993, 48],
 [8.100000000000001, 5],
 [3.6999999999999993, 27],
 [1.700000000000001, 94],
 [7.4, 93],
 [1.299999999999999, 77],
 [7.1, 85],
 [3.0999999999999996, 24],
 [2.799999999999999, 56],
 [3.5000000000000018, 17],
 [6.699999999999999, 41],
 [8.1, 80],
 [3.7, 89],
 [3.799999999999999, 17],
 [1.9999999999999982, 87],
 [7.899999999999999, 82],
 [0.3999999999999986, 70],
 [1.5, 6],
 [4.700000000000001, 22],
 [4.600000000000001, 2],
 [2.0, 71],
 [4.800000000000001, 53],
 [7.800000000000001, 79],
 [0.6999999999999993, 5],
 [3.3000000000000007, 33],
 [1.1999999999999993, 87],
 [6.999999999999998, 72],
 [0.5, 68],
 [1.3000000000000007, 29],
 [6.299999999999999, 30],
 [5.6, 44],
 [2.1000000000000014, 80],
 [0.6999999999999993, 59],
 [0.8999999999999986, 50],
 [3.4000000000000004, 82],
 [5.6, 5],
 [5.9000000000000